# Book recommender system - Graphlab

The objective of this capstone project is to build a book recomendation system.
There are many tools in python for recomendation Systems. I will be utilizing Graphlab, https://turi.com/learn/userguide/ to build a hybrid recommendation system.

There are usually two methods to build recomendtion systems.Collaborative and Content filtering approach.
Collaborative filtering: relies on the users collaborating to rate an item/product and will only be able to recommend an item if the user is already in the database and his/her preferences/ratings are recorded in the system. similarly a new item in content based approach would not have a rating and thus the system would not be able to make recommendations.

the alternative approach is to use a mix of the two approaches to build a recommendation system that selects the best method to recommend when information is not availiable from the other systems.

The dataset is Amazon books review database, obtained from http://jmcauley.ucsd.edu/data/amazon/ with permission from the authors of Image-based recommendations on styles and substitutes J. McAuley, C. Targett, J. Shi, A. van den Hengel SIGIR, 2015.


In [1]:
import graphlab
import pandas as pd
import numpy as np

In [ ]:
#gratab sframe had problems parsing some columns will use pandas read_readcsv and then select features to sframe
reviews=graphlab.SFrame.read_csv("hundredKreviews.csv")

In [ ]:
reviews.head(2)

In [21]:
reviews.show()

Canvas is accessible via web browser at the URL: http://localhost:61380/index.html
Opening Canvas in default web browser.


In [27]:
booksdb=pd.read_csv("booksinfo.csv")

In [86]:
booksdb.head(2)

,asin,title,description,brand
0,0001048791,"The Crucible: Performed by Stuart Pankin, Jero...",,
1,0001048775,Measure for Measure: Complete &amp; Unabridged,William Shakespeare is widely regarded as the ...,


In [31]:
booksdb=booksdb.drop(["Unnamed: 0","categories","imUrl","related","salesRank",price"],axis=1)
booksdb.fillna(" ", inplace=True)
booksdb=graphlab.SFrame(booksdb)

In [94]:
reviews.shape

(99985, 10)

In [96]:
booksdb.shape

(2370585, 4)

In [99]:
reviews.show()

Canvas is accessible via web browser at the URL: http://localhost:61380/index.html
Opening Canvas in default web browser.


In [107]:
reviews=reviews.join(booksdb,on='asin',how='inner')

In [171]:
reviews.head(0)

Columns:
	X1	int
	reviewerID	str
	asin	str
	reviewerName	str
	helpful	array
	unixReviewTime	int
	reviewText	str
	overall	float
	reviewTime	str
	summary	str
	title	str
	description	str
	brand	str

Rows: 0

Data:
	[]

# Build a recommender

In [109]:
model = graphlab.recommender.create(reviews, user_id='reviewerID', item_id='title')

Recsys training: model = item_similarity

Warning: Ignoring columns X1, asin, reviewerName, helpful, unixReviewTime, reviewText, overall, reviewTime, summary, description, brand;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 135.336ms                           | 100              | 2637            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.144923s

In [123]:
similar_items=model.get_similar_items()
similar_items.head(10)

title,similar,score,rank
Master Georgie,Breakfast on Pluto,0.0571428537369,1
Master Georgie,Arcadia,0.0454545617104,2
Master Georgie,BRIXTON BEACH,0.0454545617104,3
Master Georgie,N. C. Wyeth: A Biography,0.0434782505035,4
Master Georgie,Inventing Memory: A Novelof Mothers and Daughters ...,0.0434782505035,5
Master Georgie,Without Reservation: TheMaking of America's Most ...,0.0400000214577,6
Master Georgie,Catholics,0.0400000214577,7
Master Georgie,Fanny: A Fiction,0.0400000214577,8
Master Georgie,Andrew Wyeth: A SecretLife ...,0.0384615659714,9
Master Georgie,The Cloud Sketcher,0.0384615659714,10


In [131]:
similar_items.print_rows (50,4)

+-------------------------------+--------------------------------+
|             title             |            similar             |
+-------------------------------+--------------------------------+
|         Master Georgie        |       Breakfast on Pluto       |
|         Master Georgie        |            Arcadia             |
|         Master Georgie        |         BRIXTON BEACH          |
|         Master Georgie        |    N. C. Wyeth: A Biography    |
|         Master Georgie        | Inventing Memory: A Novel ...  |
|         Master Georgie        | Without Reservation: The M...  |
|         Master Georgie        |           Catholics            |
|         Master Georgie        |        Fanny: A Fiction        |
|         Master Georgie        |  Andrew Wyeth: A Secret Life   |
|         Master Georgie        |       The Cloud Sketcher       |
| The Greatest Book on &quot... |     The Book of Revelation     |
| The Berenstains' B Book (B... | Happy Haunting, Amelia Bed..

In [127]:
#Similarity graph from my preference to others
similarity_graph=graphlab.SGraph().add_edges(similar_items,src_field='title',dst_field='similar')
similarity_graph.summary()

{'num_edges': 23102, 'num_vertices': 2595}

In [128]:
shortestpath=graphlab.shortest_path.create(similarity_graph,"The Prophet")

+----------------------------+

| Number of vertices updated |

+----------------------------+

| 2467                       |

| 2155                       |

| 243                        |

| 16                         |

| 0                          |

+----------------------------+

In [132]:
shortestpath.get_path('Deafening')

[('The Prophet', 0.0),
 ('A Farewell to Arms (A Scribner Classic)', 1.0),
 ('The Bridge of San Luis Rey (Perennial Classics)', 2.0),
 ('Deafening', 3.0)]

In [90]:
#Get recommendations for first 5 users and print them
#users = range(1,6) specifies user ID of first 5 users, #k=5 specifies top 5 recommendations to be given
popularity_recomm = popularity_model.recommend(users=range(1,6),k=5)
popularity_recomm.print_rows(num_rows=25)

+------------+------------+-------+------+
| reviewerID |    asin    | score | rank |
+------------+------------+-------+------+
|     1      | 000617681X |  5.0  |  1   |
|     1      | 0006151329 |  5.0  |  2   |
|     1      | 0004723023 |  5.0  |  3   |
|     1      | 0002250357 |  5.0  |  4   |
|     1      | 0001473727 |  5.0  |  5   |
|     2      | 000617681X |  5.0  |  1   |
|     2      | 0006151329 |  5.0  |  2   |
|     2      | 0004723023 |  5.0  |  3   |
|     2      | 0002250357 |  5.0  |  4   |
|     2      | 0001473727 |  5.0  |  5   |
|     3      | 000617681X |  5.0  |  1   |
|     3      | 0006151329 |  5.0  |  2   |
|     3      | 0004723023 |  5.0  |  3   |
|     3      | 0002250357 |  5.0  |  4   |
|     3      | 0001473727 |  5.0  |  5   |
|     4      | 000617681X |  5.0  |  1   |
|     4      | 0006151329 |  5.0  |  2   |
|     4      | 0004723023 |  5.0  |  3   |
|     4      | 0002250357 |  5.0  |  4   |
|     4      | 0001473727 |  5.0  |  5   |
|     5    

# Topic Modeling: unsupervised  clustering, cluster similar documents together

A topic model assumes each document is a mixture of a set of topics, where for each topic some words are more likely than others. One statistical approach to do this is called a “topic model”. This method learns a topic model for the given document collection.

graphlab.topic_model.create(dataset, num_topics=10, initial_topics=None, alpha=None, beta=0.1, num_iterations=10, num_burnin=5, associations=None, verbose=False, print_interval=10, validation_set=None, method='auto')
.

Parameters:	
dataset : SArray of type dict or SFrame with a single column of type dict

A bag of words representation of a document corpus. Each element is a dictionary representing a single document, where the keys are words and the values are the number of times that word occurs in that document.

In [148]:
textreview=reviews["reviewText"]

In [195]:
textreview=graphlab.SArray(reviews["reviewText"])

In [197]:
docs_vec=graphlab.text_analytics.count_words(textreview)
docs_vec.dict_trim_by_keys(graphlab.text_analytics.stopwords(lang='en'), True)
docs_tfidf = graphlab.text_analytics.tf_idf(docs_vec)

In [ ]:
m=graphlab.topic_model.create(docs_tfidf,num_iterations=6,num_topics=6)
m.get_topics([5],num_words=10).print_rows()


In [ ]:
pred=m.predict(docs_tfidf)
pred.show()

In [204]:
modelbase=graphlab.recommender.item_similarity_recommender.create(reviews,user_id="reviewerID",item_id="asin",target="overall")
print 'Training RMSE', modelbase.get("training_rmse")
print 'Test RMSE', graphlab.evaluation.rmse(test['overall'],modelbase.predict(test))

Recsys training: model = item_similarity

Warning: Ignoring columns X1, reviewerName, helpful, unixReviewTime, reviewText, reviewTime, summary, title, description, brand;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 99985 observations with 68123 users and 2641 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.175995s

Training RMSE None
Test RMSE 4.3320616433


# matrix factorization - to predict the next purchase

In [150]:
train,test = reviews.random_split(0.9)

In [206]:
#Build a matrix factorization recommender to predict star rating:
model1=graphlab.factorization_recommender.create(train,target='overall',side_data_factorization=True, item_id="asin",user_id='title')

#compute train and test accuracy:
print 'Training RMSE', model1.get("training_rmse")
print 'Test RMSE', graphlab.evaluation.rmse(test['overall'],model.predict(test))

Recsys training: model = factorization_recommender

Preparing data set.

Data has 90062 observations with 2637 users and 2641 items.

Data prepared in: 1.17499s

Training factorization_recommender for recommendations.

...

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 1.25533

Final training RMSE: 1.12042

Training RMSE 1.12041630822
Test RMSE 1.0051544861


In [205]:
#Build a matrix factorization recommender to predict star rating:
model0=graphlab.factorization_recommender.create(train,target='overall',side_data_factorization=False,item_id="asin",user_id="reviewerID")

#compute train and test accuracy:
print 'Training RMSE', model0.get("training_rmse")
print 'Test RMSE', graphlab.evaluation.rmse(test['overall'],model.predict(test))


Recsys training: model = factorization_recommender

Preparing data set.

Data has 90062 observations with 62678 users and 2641 items.

Data prepared in: 1.28541s

Training factorization_recommender for recommendations.

...

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.185147

Final training RMSE: 0.430286

Training RMSE 0.430285754327
Test RMSE 1.0051544861


# Conclusion:

including more features and combining the collaborative and content based matices... 
was better interms of recommending better book recommendations. Though the initial question was to buils "a better Recommender system
"it was clear from the models that there are model differences and its in our best interest to look for the best systems.

# Recommendation:

To improve recommendation and cold start problem the following areas could be explored.
featurized matrix factorization (time of the day, items bought together to predict users rating for unrated books)
boosting...blending models.